In [1]:
import numpy as np
import pandas as pd

from ols_bootstrap.pairs import PairsBootstrap
from ols_bootstrap.residual import ResidualBootstrap
from ols_bootstrap.wild import WildBootstrap

In [2]:
df = pd.read_csv('./balance2018.csv')
df = df[df['sales_clean'] != 0]
df = df[['sales_clean', 'tanass_clean', 'tax']]
df = df.dropna(subset=['tanass_clean', 'tax'])

df_scaled = df.applymap(lambda x: np.log(x + 1))

/home/phamv/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_sample = df_scaled.sample(n=10000, replace=False)

Y_data = pd.DataFrame(df_sample.iloc[:, 0])
X_data = pd.DataFrame(df_sample.iloc[:, 1:])

## Default SE on the original OLS is HC3, default CI on bootstrapped parameter is BC. 

In [4]:
psb = PairsBootstrap(Y_data, X_data, reps = 1000)
psb.fit()

In [5]:
rsb = ResidualBootstrap(Y_data, X_data, reps = 1000)
rsb.fit()

In [6]:
wb_stdn = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "standard_normal")
wb_stdn.fit()

In [7]:
wb_rad = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "rademacher")
wb_rad.fit()

In [8]:
wb_mam = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "mammen")
wb_mam.fit()

In [9]:
psb.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                     Pairs Bootstrap results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                      |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2298   |      6.2291      | 0.0007 |     0.0443    |    0.0438    |     1.06     | [6.1412, 6.3133] |  0.1721 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1673   |      0.1672      | 0.0001 |     0.0050 

In [10]:
rsb.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                    Residual Bootstrap results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                    |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2298   |      6.2318      | 0.0019 |     0.0443    |    0.0350    |    21.02     | [6.1585, 6.2971] |  0.1386 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1673   |      0.1670      | 0.0002 |     0.0050 

In [11]:
wb_stdn.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|           Wild Bootstrap with Standard Normal results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI            |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2298   |      6.2289      | 0.0009 |     0.0443    |    0.0338    |    23.77     | [6.1683, 6.2982] |  0.1299 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1673   |      0.1673      | 0.0000 |     0.0050 

In [12]:
wb_rad.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|              Wild Bootstrap with Rademacher results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI              |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2298   |      6.2290      | 0.0008 |     0.0443    |    0.0353    |    20.21     | [6.1669, 6.3022] |  0.1352 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1673   |      0.1674      | 0.0001 |     0.0050 

In [13]:
wb_mam.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                Wild Bootstrap with Mammen results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2298   |      6.2291      | 0.0007 |     0.0443    |    0.0345    |    22.20     | [6.1628, 6.2979] |  0.1351 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1673   |      0.1673      | 0.0000 |     0.0050 

## Some useful methods were implemented (let's use it on wb_mam object of Wild Bootstrap with Mammen)

- The common in the following three methods is that either a string (if one wishes to capture one variable or ci) or 1-D like array can be provided to 'which_var' and/or 'which_ci' (the latter if exists in that object class).

### get_ci() method

#### Vanila version is when only the actual CI was used with all independent variables

In [14]:
wb_mam.get_ci()

bc        
                 lwb     upb
const         6.1628  6.2979
tanass_clean  0.1599  0.1749
tax           0.5136  0.5399

#### However, any combination of ('bc', 'bca', 'percentile) CI types could be selected and any combinations of independent variables can be chosen with 'which_ci' and 'which_var' optional arguments, respectively.

Please note that if choosing 'bca' the calculation can take a while as it uses jacknife resampling for calculating the acceleration factor

In [15]:
wb_mam.get_ci(which_ci=['bc', 'percentile'], which_var=['tax', 'const'])

bc         percentile        
          lwb     upb        lwb     upb
tax    0.5136  0.5399     0.5136  0.5399
const  6.1628  6.2979     6.1613  6.2961

In [22]:
wb_mam.get_ci(which_ci='bca', which_var='tanass_clean')

bca       
                 lwb    upb
tanass_clean  0.1599  0.175

### get_all_se() method

The following SE-s are calculated: 
- bootstrapped - standard error of the bootstrapped parameters
- constant - model-based OLS Standard Errors, that is, constant variance is assumed 
- HC0, HC1, HC2, HC3, HC4, HC4m, HC5 - Heteroskedasticity-Consistent Standard Errors (HCE) using sandwich estimators 

#### Vanila version is when using all indepencdent variables. 

In [16]:
wb_mam.get_all_se()

,bootstrapped,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
const,0.0345,0.0348,0.0443,0.0443,0.0443,0.0443,0.0443,0.0443,0.0443
tanass_clean,0.0038,0.0039,0.0050,0.0050,0.0050,0.0050,0.0050,0.0050,0.0050
tax,0.0067,0.0065,0.0090,0.0090,0.0090,0.0091,0.0091,0.0091,0.0091


#### A subset of indepent variables can be chosen with 'which_var' argument to calculate the above-mentioned 9 SE-s

In [17]:
wb_mam.get_all_se(which_var=['tanass_clean', 'tax'])

,bootstrapped,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
tanass_clean,0.0038,0.0039,0.005,0.005,0.005,0.0050,0.0050,0.0050,0.0050
tax,0.0067,0.0065,0.009,0.009,0.009,0.0091,0.0091,0.0091,0.0091


In [21]:
wb_mam.get_all_se(which_var='tax')

,bootstrapped,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
tax,0.0067,0.0065,0.009,0.009,0.009,0.0091,0.0091,0.0091,0.0091


### get_bootstrap_params() method

#### Vanila version: Returning a dataframe capturing the parameter estimate of ALL each independent variables in each (wild) bootstrap.

In [18]:
wb_mam.get_bootstrap_params()

,const,tanass_clean,tax
0,6.240260,0.165927,0.528699
1,6.309473,0.166976,0.512988
2,6.190222,0.165459,0.542963
3,6.265918,0.162811,0.533156
4,6.215660,0.166600,0.527498
...,...,...,...
995,6.267102,0.159051,0.534078
996,6.179996,0.163865,0.534031
997,6.285530,0.169895,0.509611
998,6.234030,0.167179,0.529287


#### As usual, the desired independent variable can be chosen with 'which_var' argument

In [19]:
wb_mam.get_bootstrap_params(which_var='tax')

,tax
0,0.528699
1,0.512988
2,0.542963
3,0.533156
4,0.527498
...,...
995,0.534078
996,0.534031
997,0.509611
998,0.529287


In [20]:
wb_mam.get_bootstrap_params(which_var=('const', 'tanass_clean'))

,const,tanass_clean
0,6.240260,0.165927
1,6.309473,0.166976
2,6.190222,0.165459
3,6.265918,0.162811
4,6.215660,0.166600
...,...,...
995,6.267102,0.159051
996,6.179996,0.163865
997,6.285530,0.169895
998,6.234030,0.167179
